# Capítulo 1: Introdução ao Apache Iceberg

Este notebook demonstra os conceitos fundamentais do Apache Iceberg e sua integração com DuckDB.

## 1. Setup e Configuração

In [ ]:
import duckdb
import sys

# Configurar encoding UTF-8 para Windows
if sys.platform == 'win32':
    sys.stdout.reconfigure(encoding='utf-8')

# Criar conexão DuckDB
con = duckdb.connect()

# Instalar e carregar extensão Iceberg
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")

print("✅ DuckDB configurado com Iceberg")

## 2. O que é Apache Iceberg?

Apache Iceberg é um **formato de tabela aberto** para grandes datasets analíticos que oferece:

- ✅ **ACID transactions**: Garantias de atomicidade, consistência, isolamento e durabilidade
- ✅ **Schema evolution**: Evolução de schema sem reescrever dados
- ✅ **Time travel**: Acesso a versões históricas dos dados
- ✅ **Partition evolution**: Mudança de estratégia de particionamento
- ✅ **Hidden partitioning**: Particionamento transparente para usuários

## 3. Iceberg vs Parquet

### Problemas com formatos tradicionais (Parquet/CSV):

- ❌ Sem controle de versão
- ❌ Sem transações ACID
- ❌ Schema fixo
- ❌ Particionamento manual e propenso a erros
- ❌ Problemas com leituras concorrentes

### Soluções do Iceberg:

- ✅ Versionamento completo (snapshots)
- ✅ Transações ACID garantidas
- ✅ Schema evolution sem downtime
- ✅ Particionamento automático e evoluível
- ✅ Leituras isoladas de escritas

## 4. Arquitetura do Iceberg

```
┌─────────────────────────────────────┐
│        CATALOG (REST/Glue)          │  ← Gerencia metadados
└─────────────────────────────────────┘
              ↓
┌─────────────────────────────────────┐
│     METADATA FILES (.json)          │  ← Schema, partições, snapshots
└─────────────────────────────────────┘
              ↓
┌─────────────────────────────────────┐
│     MANIFEST LISTS (.avro)          │  ← Lista de manifests
└─────────────────────────────────────┘
              ↓
┌─────────────────────────────────────┐
│     MANIFEST FILES (.avro)          │  ← Lista de data files
└─────────────────────────────────────┘
              ↓
┌─────────────────────────────────────┐
│     DATA FILES (.parquet)           │  ← Dados reais
└─────────────────────────────────────┘
```

## 5. Conceitos Fundamentais

### Snapshots
Cada modificação na tabela cria um novo **snapshot** (foto do estado):

```sql
-- Versão 1: Tabela criada com 1000 linhas
CREATE TABLE vendas ...
INSERT INTO vendas ... (1000 rows)
-- Snapshot ID: 12345

-- Versão 2: Adicionadas 500 linhas
INSERT INTO vendas ... (500 rows)
-- Snapshot ID: 12346
```

### Schema Evolution

Modificar schema sem reescrever dados:

```sql
-- Adicionar coluna (sem reescrever arquivos existentes)
ALTER TABLE customers ADD COLUMN email VARCHAR;

-- Renomear coluna (apenas metadados)
ALTER TABLE customers RENAME COLUMN name TO full_name;
```

### Hidden Partitioning

**Problema tradicional (Hive-style):**
```sql
-- Usuário precisa especificar partição
SELECT * FROM sales WHERE year = 2024 AND month = 1;
```

**Solução Iceberg:**
```sql
-- Particionamento transparente
SELECT * FROM sales WHERE sale_date = '2024-01-15';
-- Iceberg automaticamente usa partições corretas!
```

## 6. Por que DuckDB + Iceberg?

### DuckDB oferece:
- 🚀 Performance extrema para OLAP
- 💾 Processamento in-memory eficiente
- 🐍 Integração perfeita com Python/Pandas
- 📦 Zero dependências (embedded database)

### Iceberg oferece:
- 📊 Gerenciamento de tabelas em escala
- 🕐 Time travel e versionamento
- 🔄 ACID transactions
- 🌐 Interoperabilidade (Spark, Flink, Trino)

## 7. Exemplo Conceitual de Código

In [ ]:
# Exemplo conceitual (não executa sem tabela Iceberg)
exemplo_codigo = """
import duckdb

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")

# Query em tabela Iceberg
result = con.execute('''
    SELECT category, sum(revenue)
    FROM iceberg_scan('s3://bucket/sales')
    GROUP BY category
''').df()
"""

print("Exemplo de código DuckDB + Iceberg:")
print(exemplo_codigo)

## 8. Terminologia Essencial

| Termo | Definição |
|-------|----------|
| **Snapshot** | Versão imutável da tabela em um momento específico |
| **Manifest** | Lista de arquivos de dados com metadados (min/max, contagens) |
| **Manifest List** | Lista de manifests que compõem um snapshot |
| **Data File** | Arquivo Parquet/Avro/ORC com dados reais |
| **Metadata File** | Arquivo JSON com schema, partições e histórico |
| **Catalog** | Sistema que rastreia tabelas Iceberg e seus metadados |
| **Partition Spec** | Definição de como dados são particionados |
| **Schema** | Definição de colunas e tipos da tabela |

## 9. Quando Usar Iceberg?

### ✅ Use Iceberg Quando:
- Precisa de transações ACID
- Múltiplos writers concorrentes
- Schema muda com frequência
- Precisa de time travel / auditoria
- Dados particionados complexos
- Interoperabilidade entre engines
- Lakehouse architecture

### ❌ Não Precisa de Iceberg Quando:
- Dados pequenos (< 1 GB)
- Read-only workloads simples
- Single writer, no concurrency
- Schema nunca muda
- Não precisa de versionamento
- Apenas análise ad-hoc rápida

## 10. Exercícios Práticos

1. Pesquise sobre outros formatos de tabela (Delta Lake, Hudi)
2. Leia a documentação oficial do Apache Iceberg
3. Explore exemplos de uso do DuckDB com Iceberg
4. Identifique casos de uso na sua organização
5. Configure um ambiente de desenvolvimento local

## Próximos Passos

No próximo capítulo, você aprenderá como instalar e configurar a extensão Iceberg no DuckDB, preparando seu ambiente para trabalhar com tabelas Iceberg.

---

**Referências**:
- [DuckDB Iceberg Extension](https://duckdb.org/docs/stable/core_extensions/iceberg/overview)
- [Apache Iceberg Official Docs](https://iceberg.apache.org/)